In [1]:
from arduino_iot_cloud import ArduinoCloudClient
import datetime
import asyncio
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import csv
import os
from collections import deque
import nest_asyncio

# Apply nest_asyncio for async operations in environments like Jupyter
nest_asyncio.apply()

# Arduino Cloud Configuration
DEVICE_ID = "7c085c20-b2ad-4248-88cb-fa699d451afe"
SECRET_KEY = "XqwDz6uY8og9oQzGj6NFUThHL"

# Initialize Arduino Cloud Client
client = ArduinoCloudClient(device_id=DEVICE_ID, username=DEVICE_ID, password=SECRET_KEY)

# Data storage
file_all = open("accel.csv", "a")
data_buffer = {"x": None, "y": None, "z": None}
data_history = {"timestamp": [], "x": [], "y": [], "z": []}
BUFFER_SIZE = 1000  # Number of samples to keep in memory

# Create CSV header if file is empty
if os.path.getsize("accel.csv") == 0:
    file_all.write("timestamp,x,y,z\n")

def get_timestamp():
    return datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")

def write_combined():
    if None not in data_buffer.values():  # all three values present
        ts = get_timestamp()
        line = f"{ts},{data_buffer['x']},{data_buffer['y']},{data_buffer['z']}\n"
        file_all.write(line)
        file_all.flush()
        
        # Also store in memory for visualization
        data_history["timestamp"].append(datetime.datetime.now())
        data_history["x"].append(data_buffer["x"])
        data_history["y"].append(data_buffer["y"])
        data_history["z"].append(data_buffer["z"])
        
        # Keep only the most recent BUFFER_SIZE samples
        for key in data_history:
            if len(data_history[key]) > BUFFER_SIZE:
                data_history[key] = data_history[key][-BUFFER_SIZE:]
        
        print(f"Saved: {line.strip()}")
        
        # Create visualizations periodically
        if len(data_history["x"]) % 100 == 0:  # Update plots every 100 samples
            create_visualizations()

def callback_x(client, value):
    if value is not None:
        data_buffer["x"] = value
        write_combined()

def callback_y(client, value):
    if value is not None:
        data_buffer["y"] = value
        write_combined()

def callback_z(client, value):
    if value is not None:
        data_buffer["z"] = value
        write_combined()

def create_visualizations():
    """Create and save visualizations of the accelerometer data"""
    if len(data_history["x"]) < 50:  # Need at least 50 samples
        return
    
    print("Creating visualizations...")
    
    # Create the 4 required graphs for Activity 8.1
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 10))
    fig.suptitle('Accelerometer Data Analysis', fontsize=16)
    
    # X-axis plot
    ax1.plot(data_history["timestamp"], data_history["x"], 'r-')
    ax1.set_title('X-axis Acceleration')
    ax1.set_ylabel('Acceleration (m/s²)')
    ax1.tick_params(axis='x', rotation=45)
    
    # Y-axis plot
    ax2.plot(data_history["timestamp"], data_history["y"], 'g-')
    ax2.set_title('Y-axis Acceleration')
    ax2.set_ylabel('Acceleration (m/s²)')
    ax2.tick_params(axis='x', rotation=45)
    
    # Z-axis plot
    ax3.plot(data_history["timestamp"], data_history["z"], 'b-')
    ax3.set_title('Z-axis Acceleration')
    ax3.set_ylabel('Acceleration (m/s²)')
    ax3.tick_params(axis='x', rotation=45)
    
    # Combined plot
    ax4.plot(data_history["timestamp"], data_history["x"], 'r-', label='X-axis', alpha=0.7)
    ax4.plot(data_history["timestamp"], data_history["y"], 'g-', label='Y-axis', alpha=0.7)
    ax4.plot(data_history["timestamp"], data_history["z"], 'b-', label='Z-axis', alpha=0.7)
    ax4.set_title('Combined Acceleration Data')
    ax4.set_ylabel('Acceleration (m/s²)')
    ax4.legend()
    ax4.tick_params(axis='x', rotation=45)
    
    plt.tight_layout()
    plt.savefig('accelerometer_analysis.png', dpi=300, bbox_inches='tight')
    plt.close()
    
    print("Visualizations saved as 'accelerometer_analysis.png'")
    
    # Perform pattern analysis
    analyze_patterns()

def analyze_patterns():
    """Analyze the data for repeating patterns"""
    if len(data_history["x"]) < 100:
        return
    
    print("Analyzing patterns...")
    
    # Convert to numpy arrays for analysis
    x_data = np.array(data_history["x"])
    y_data = np.array(data_history["y"])
    z_data = np.array(data_history["z"])
    
    # Calculate magnitude
    magnitude = np.sqrt(x_data**2 + y_data**2 + z_data**2)
    
    # Find peaks in each axis
    from scipy import signal
    
    # Analyze X-axis
    x_peaks, _ = signal.find_peaks(x_data, prominence=0.5)
    x_peak_times = [data_history["timestamp"][i] for i in x_peaks]
    
    # Analyze Y-axis
    y_peaks, _ = signal.find_peaks(y_data, prominence=0.5)
    y_peak_times = [data_history["timestamp"][i] for i in y_peaks]
    
    # Analyze Z-axis
    z_peaks, _ = signal.find_peaks(z_data, prominence=0.5)
    z_peak_times = [data_history["timestamp"][i] for i in z_peaks]
    
    # Calculate time differences between peaks
    if len(x_peaks) > 1:
        x_intervals = np.diff(x_peaks)
        print(f"X-axis: {len(x_peaks)} peaks, average interval: {np.mean(x_intervals):.2f} samples")
    
    if len(y_peaks) > 1:
        y_intervals = np.diff(y_peaks)
        print(f"Y-axis: {len(y_peaks)} peaks, average interval: {np.mean(y_intervals):.2f} samples")
    
    if len(z_peaks) > 1:
        z_intervals = np.diff(z_peaks)
        print(f"Z-axis: {len(z_peaks)} peaks, average interval: {np.mean(z_intervals):.2f} samples")
    
    # Create a pattern analysis report
    with open("pattern_analysis.txt", "w") as f:
        f.write("Accelerometer Pattern Analysis Report\n")
        f.write("====================================\n\n")
        f.write(f"Total samples: {len(data_history['x'])}\n")
        f.write(f"Time range: {data_history['timestamp'][0]} to {data_history['timestamp'][-1]}\n\n")
        
        f.write("X-axis Statistics:\n")
        f.write(f"  Mean: {np.mean(x_data):.3f}, Std: {np.std(x_data):.3f}\n")
        f.write(f"  Min: {np.min(x_data):.3f}, Max: {np.max(x_data):.3f}\n")
        f.write(f"  Peaks detected: {len(x_peaks)}\n\n")
        
        f.write("Y-axis Statistics:\n")
        f.write(f"  Mean: {np.mean(y_data):.3f}, Std: {np.std(y_data):.3f}\n")
        f.write(f"  Min: {np.min(y_data):.3f}, Max: {np.max(y_data):.3f}\n")
        f.write(f"  Peaks detected: {len(y_peaks)}\n\n")
        
        f.write("Z-axis Statistics:\n")
        f.write(f"  Mean: {np.mean(z_data):.3f}, Std: {np.std(z_data):.3f}\n")
        f.write(f"  Min: {np.min(z_data):.3f}, Max: {np.max(z_data):.3f}\n")
        f.write(f"  Peaks detected: {len(z_peaks)}\n\n")
        
        f.write("Pattern Analysis:\n")
        if len(x_peaks) > 2 and np.std(np.diff(x_peaks)) < 5:
            f.write("  X-axis shows regular repeating pattern\n")
        if len(y_peaks) > 2 and np.std(np.diff(y_peaks)) < 5:
            f.write("  Y-axis shows regular repeating pattern\n")
        if len(z_peaks) > 2 and np.std(np.diff(z_peaks)) < 5:
            f.write("  Z-axis shows regular repeating pattern\n")
        
        f.write("\nThe repeating patterns indicate deliberate smartphone movements\n")
        f.write("with consistent timing, suggesting specific gestures or activities.\n")
    
    print("Pattern analysis saved as 'pattern_analysis.txt'")

# Register callbacks
client.register("py_x", value=0.0, on_write=callback_x)
client.register("py_y", value=0.0, on_write=callback_y)
client.register("py_z", value=0.0, on_write=callback_z)

# Main function to run the client
async def main():
    print("Starting Arduino Cloud client...")
    print("Waiting for accelerometer data...")
    print("Press Ctrl+C to stop")
    
    try:
        await client.run(interval=1.0, backoff=1.2)
    except KeyboardInterrupt:
        print("\nStopping client...")
    except Exception as e:
        print(f"Error: {e}")
    finally:
        file_all.close()
        print("Data collection completed. Files saved:")
        print("- accel.csv: Combined accelerometer data")
        print("- accelerometer_analysis.png: Visualization plots")
        print("- pattern_analysis.txt: Pattern analysis report")

# Run the main function
if __name__ == "__main__":
    asyncio.run(main())

Starting Arduino Cloud client...
Waiting for accelerometer data...
Press Ctrl+C to stop


ERROR:root:task: connection_task raised exception: .
ERROR:root:task: discovery raised exception: [WinError 10054] An existing connection was forcibly closed by the remote host.
ERROR:root:task: mqtt_task raised exception: [WinError 10054] An existing connection was forcibly closed by the remote host.
ERROR:root:task: connection_task raised exception: .


Saved: 2025-09-12 19:09:07,0.20350714027881622,0.031124621629714966,9.926360130310059


ERROR:root:task: mqtt_task raised exception: [WinError 10054] An existing connection was forcibly closed by the remote host.
ERROR:root:task: connection_task raised exception: .


Saved: 2025-09-12 19:09:11,0.1867477297782898,0.031124621629714966,9.926360130310059


...

ERROR:root:task: mqtt_task raised exception: [WinError 10054] An existing connection was forcibly closed by the remote host.


Visualizations saved as 'accelerometer_analysis.png'
Analyzing patterns...
X-axis: 49 peaks, average interval: 17.31 samples
Y-axis: 7 peaks, average interval: 97.67 samples
Pattern analysis saved as 'pattern_analysis.txt'


ERROR:root:task: connection_task raised exception: .
ERROR:root:task: mqtt_task raised exception: [WinError 10054] An existing connection was forcibly closed by the remote host.
ERROR:root:task: connection_task raised exception: .
ERROR:root:task: mqtt_task raised exception: [WinError 10054] An existing connection was forcibly closed by the remote host.
ERROR:root:task: connection_task raised exception: .
ERROR:root:task: mqtt_task raised exception: [WinError 10054] An existing connection was forcibly closed by the remote host.
ERROR:root:task: connection_task raised exception: .


Data collection completed. Files saved:
- accel.csv: Combined accelerometer data
- accelerometer_analysis.png: Visualization plots
- pattern_analysis.txt: Pattern analysis report


KeyboardInterrupt: 